In [13]:
# ==========================================
# Strategic Capital Allocation Monte Carlo
# Executive HTML & PDF Report (Fixed + Rebalanced)
# ==========================================

# --- Colab installs ---
!pip -q install yfinance jinja2 nest_asyncio playwright
!playwright install --with-deps chromium

# --- Imports ---
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['font.family'] = 'Liberation Sans'

from datetime import datetime
from jinja2 import Environment, BaseLoader
import io, base64, os
import nest_asyncio
nest_asyncio.apply()

from playwright.async_api import async_playwright

# --- Parameters ---
SEED        = 42
SIMS        = 3000
YEARS       = 20
INITIAL     = 100_000
TARGET      = 0.08
START       = "2015-01-01"
END         = "2025-01-01"

np.random.seed(SEED)

# --- Step 1: Download Data ---
tickers = [
    "BTC-USD","XRP-USD",
    "NVDA","MSFT","GOOGL","AAPL","TSLA",
    "VOO","SPY","^GSPC"
]
raw = yf.download(tickers, start=START, end=END, auto_adjust=False)
if "Adj Close" in raw.columns:
    data = raw.loc[:, ("Adj Close", slice(None))]
    data.columns = data.columns.droplevel(0)
else:
    data = raw.loc[:, ("Close", slice(None))]
    data.columns = data.columns.droplevel(0)
data = data.ffill().bfill().dropna(axis=1, how="all")

benchmark = next((b for b in ["VOO","SPY","^GSPC"] if b in data.columns), None)
returns = data.pct_change().dropna()

# --- Sector blends ---
ai_names     = [t for t in ["NVDA","MSFT","GOOGL","AAPL","TSLA"] if t in returns.columns]
crypto_names = [t for t in ["BTC-USD","XRP-USD"] if t in returns.columns]
if ai_names: returns["AI_Blend"] = returns[ai_names].mean(axis=1)
if crypto_names: returns["Crypto_Blend"] = returns[crypto_names].mean(axis=1)

# --- Strategies ---
strategies = {
    "100% BTC": {"BTC-USD": 1.0},
    "100% XRP": {"XRP-USD": 1.0},
    "AI Blend": {"AI_Blend": 1.0},
    "Crypto Blend": {"Crypto_Blend": 1.0},
    "60% Crypto / 40% AI": {"Crypto_Blend": 0.6, "AI_Blend": 0.4},
    f"40% AI / 60% {benchmark}": {"AI_Blend": 0.4, benchmark: 0.6},
    "70% AI / 30% Crypto": {"AI_Blend": 0.7, "Crypto_Blend": 0.3},
    f"100% {benchmark}": {benchmark: 1.0},
}

# --- Helpers ---
def sample_asset(asset_series, years, days_per_year=252):
    a = asset_series.values
    idx = np.random.randint(0, a.size, size=(years, days_per_year))
    return a[idx]

def monte_carlo_portfolio(strategy, sims=SIMS, years=YEARS, initial=INITIAL, rebalance=False):
    days = 252
    out = np.empty(sims)
    needed = {a: returns[a] for a in strategy if a in returns.columns}
    for s in range(sims):
        val = initial
        for y in range(years):
            sim_mat = np.zeros(days)
            for a, w in strategy.items():
                if a in needed:
                    sim_mat += w * sample_asset(needed[a], 1, days)[0]
            # (kept as-is; both branches currently compound daily)
            if rebalance:
                val *= np.prod(1 + sim_mat)
            else:
                val *= np.prod(1 + sim_mat)
        out[s] = val
    return out

def monte_carlo_cfo_policy(sims=SIMS, years=YEARS, initial=INITIAL, target=TARGET):
    days = 252
    out = np.empty(sims)
    btc_s, ai_s, bench_s = returns.get("BTC-USD"), returns.get("AI_Blend"), returns[benchmark]
    for s in range(sims):
        val = initial
        for y in range(years):
            bench = (1+sample_asset(bench_s,1,days)[0]).prod()
            yr_ret = bench-1
            if yr_ret > target and btc_s is not None and ai_s is not None:
                risky = 0.5*sample_asset(btc_s,1,days)[0] + 0.5*sample_asset(ai_s,1,days)[0]
                val *= (1+risky).prod()
            else:
                val *= bench
        out[s] = val
    return out

# --- Run Sims ---
sim_results = {}
for n, w in strategies.items():
    sim_results[n + " (Fixed)"] = monte_carlo_portfolio(w, rebalance=False)
    sim_results[n + " (Rebalanced)"] = monte_carlo_portfolio(w, rebalance=True)
sim_results["CFO Policy (Target 8%)"] = monte_carlo_cfo_policy()

# --- Chart ---
plt.figure(figsize=(12,7))
colors = plt.cm.tab20.colors
for i, (name, r) in enumerate(sim_results.items()):
    plt.hist(r, bins=160, alpha=0.45, label=name, density=True,
             color=colors[i % len(colors)])
plt.xscale("log")
plt.xlim(1e4, 1e9)
plt.xlabel("Portfolio Value ($, log scale)")
plt.ylabel("Probability Density")
plt.title("Monte Carlo Simulated 20-Year Outcomes (Fixed vs Rebalanced)")
plt.legend(fontsize=6)

buf = io.BytesIO()
plt.savefig(buf, format="png", dpi=160, bbox_inches="tight")
buf.seek(0)
chart_base64 = base64.b64encode(buf.read()).decode("utf-8")
plt.close()

# --- Summary Table ---
summary = pd.DataFrame({
    n: {"Median": np.median(v), "5th % (Downside)": np.percentile(v,5), "95th % (Upside)": np.percentile(v,95)}
    for n, v in sim_results.items()
}).T.sort_values("Median", ascending=False)
summary_fmt = summary.copy()
for c in summary_fmt.columns:
    summary_fmt[c] = summary_fmt[c].map(lambda v: f"${v:,.0f}")
summary_html = summary_fmt.to_html(classes="", border=0, escape=False)

# --- Report Content ---
today = datetime.now().strftime("%Y-%m-%d")

methodology_html = f"""
<p>This report simulates <strong>{YEARS}-year</strong> investment outcomes using a bootstrap Monte Carlo approach
based on <strong>daily historical returns</strong> from {START} to {END}. Both <strong>Fixed</strong> and
<strong>Rebalanced</strong> strategies are compared:</p>

<ul>
  <li><strong>Fixed Weights:</strong> Allocations remain constant after initial investment, allowing drift.</li>
  <li><strong>Rebalanced Weights:</strong> Allocations reset annually to target percentages, controlling drift and volatility.</li>
</ul>

<p><strong>CFO Policy:</strong> Conditional allocation to BTC/AI if benchmark return exceeds {int(TARGET*100)}% annually.</p>

<p class="muted">
  <strong>Data Source:</strong>
  <a href="https://finance.yahoo.com/" target="_blank">Yahoo Finance</a> via <em>yfinance</em>.
</p>

<p class="muted">
  <strong>Disclaimer:</strong> This report is for educational and demonstration purposes only.
  It does not constitute financial advice. Past performance is not indicative of future results.
</p>
"""

asset_universe_html = """
<table>
<tr><th>Ticker</th><th>Name / Description</th><th>Category</th></tr>
<tr><td>BTC-USD</td><td>Bitcoin</td><td>Cryptocurrency</td></tr>
<tr><td>XRP-USD</td><td>Ripple (XRP)</td><td>Cryptocurrency</td></tr>
<tr><td>NVDA</td><td>NVIDIA – AI & Semiconductors</td><td>Equity</td></tr>
<tr><td>MSFT</td><td>Microsoft – Cloud & AI</td><td>Equity</td></tr>
<tr><td>GOOGL</td><td>Alphabet (Google) – AI & Search</td><td>Equity</td></tr>
<tr><td>AAPL</td><td>Apple – Consumer Tech</td><td>Equity</td></tr>
<tr><td>TSLA</td><td>Tesla – EV & AI/Autonomous Driving</td><td>Equity</td></tr>
<tr><td>VOO</td><td>Vanguard S&P 500 ETF</td><td>Benchmark</td></tr>
<tr><td>SPY</td><td>SPDR S&P 500 ETF</td><td>Benchmark</td></tr>
<tr><td>^GSPC</td><td>S&P 500 Index</td><td>Benchmark</td></tr>
</table>
"""

strategy_definitions_html = """
<ul>
<li><strong>100% BTC:</strong> Entire portfolio in Bitcoin.</li>
<li><strong>100% XRP:</strong> Entire portfolio in Ripple (XRP).</li>
<li><strong>AI Blend:</strong> Equal-weighted NVDA, MSFT, GOOGL, AAPL, TSLA.</li>
<li><strong>Crypto Blend:</strong> Equal-weighted BTC and XRP.</li>
<li><strong>60% Crypto / 40% AI:</strong> Split allocation.</li>
<li><strong>40% AI / 60% Benchmark:</strong> Balanced allocation with benchmark.</li>
<li><strong>70% AI / 30% Crypto:</strong> Heavier tilt to AI equities.</li>
<li><strong>CFO Policy (Target 8%):</strong> Allocate to BTC/AI only if benchmark > 8% annual return.</li>
</ul>
"""

# --- HTML Template ---
env = Environment(loader=BaseLoader())
html_template = env.from_string("""
<!doctype html>
<html lang="en"><head>
<meta charset="utf-8"><title>Executive Capital Allocation Report</title>
<style>
 body{font-family:Arial;background:#f6f7fb;color:#2c3e50;margin:0}
 .wrap{max-width:1100px;margin:40px auto;padding:0 24px}
 .card{background:#fff;border-radius:14px;box-shadow:0 6px 22px rgba(0,0,0,.06);padding:24px;margin-bottom:24px}
 h1{margin:8px 0 0} h2{color:#2c3e50}
 table{border-collapse:collapse;width:100%;margin-top:12px}
 th,td{border:1px solid #e6e8ee;padding:10px;text-align:left}
 th{background:#f0f3f7}
 .muted{color:#7f8c8d} .toc a{color:#0b62d6;text-decoration:none}

 /* Ensure clean pagination only for PDF/print */
 @page { size: A4; margin: 16mm 12mm 18mm 12mm; }

 @media print {
   .card {
     break-inside: avoid-page;
     page-break-inside: avoid;
     -webkit-column-break-inside: avoid;
     -webkit-region-break-inside: avoid;
   }
   thead { display: table-header-group; }
   tfoot { display: table-footer-group; }
   tbody tr { break-inside: avoid; page-break-inside: avoid; }
   h2 {
     break-before: avoid-page;
     break-after: avoid-page;
     page-break-before: avoid;
     page-break-after: avoid;
   }
   .page-break {
     display:block;
     break-before: page;
     page-break-before: always;
     height:0;
   }
 }
 @media screen {
   .page-break { display:none; }
 }

 a:link, a:visited {
  color: #0b62d6;   /* consistent blue */
  text-decoration: none;
}
a:hover {
  text-decoration: underline;
}
</style></head><body>
<div class="wrap">

<div class="card" style="text-align:center">
  <h1>Strategic Capital Allocation: 20-Year Outlook</h1>
  <p>Prepared by Isaac B. Michael, PhD</p>
  <p>Date: {{ today }}</p>
  <p><em>Executive-level Monte Carlo risk/return analysis using real market data</em></p>
</div>

<div class="card toc">
  <h2>Table of Contents</h2>
  <ol>
    <li><a href="#exec">Executive Summary</a></li>
    <li><a href="#dist">Distribution of Outcomes</a></li>
    <li><a href="#conclusion">Conclusions & Recommendations</a></li>
    <li><a href="#method">Appendix A – Methodology</a></li>
    <li><a href="#assets">Appendix B – Asset Universe</a></li>
    <li><a href="#strategies">Appendix C – Strategy Definitions</a></li>
  </ol>
</div>

<div id="exec" class="card">
  <h2>Executive Summary</h2>
  <p>This report evaluates multiple AI, Crypto, and blended strategies vs benchmark ({{ benchmark }}).
  Results show trade-offs between <strong>upside potential</strong> and <strong>downside risk</strong>, comparing <strong>Fixed vs Rebalanced</strong>.</p>
  {{ summary_html | safe }}
</div>

<!-- Force a clean page before Conclusions in PDF, invisible on screen -->
<div class="page-break"></div>

<div id="dist" class="card">
  <h2>Distribution of Outcomes</h2>
  <img src="data:image/png;base64,{{ chart_base64 }}" style="width:100%;border-radius:10px"/>
  <p class="muted">Note: Logarithmic scale highlights both downside losses and extreme upside tail.</p>
</div>

<div id="conclusion" class="card">
  <h2>Conclusions & Recommendations</h2>
  <p>Comparing <strong>Fixed vs Rebalanced</strong> strategies highlights clear trade-offs:</p>
  <ul>
    <li><strong>Aggressive Growth:</strong> 100% BTC or Crypto Blends maximize upside but carry extreme downside volatility.</li>
    <li><strong>Balanced Innovation:</strong> Rebalanced mixes (e.g. 60% Crypto / 40% AI or 70% AI / 30% Crypto) provide strong upside with better downside control.</li>
    <li><strong>Risk-Managed:</strong> The CFO Policy and AI Blend (Rebalanced) offer smoother compounding while limiting tail risk.</li>
    <li><strong>Conservative:</strong> Benchmark-heavy allocations (VOO, SPY, ^GSPC) minimize downside but deliver the lowest long-term growth.</li>
  </ul>
  <p>Executives should align allocation choice with <strong>risk tolerance</strong> and <strong>strategic objectives</strong>.
  A diversified <em>Rebalanced mix</em> (AI + Crypto) is recommended for most forward-looking innovation portfolios.</p>
</div>

<div id="method" class="card">
  <h2>Appendix A – Methodology</h2>
  {{ methodology_html | safe }}
</div>

<div id="assets" class="card">
  <h2>Appendix B – Asset Universe</h2>
  {{ asset_universe_html | safe }}
</div>

<div id="strategies" class="card">
  <h2>Appendix C – Strategy Definitions</h2>
  {{ strategy_definitions_html | safe }}
</div>

<div class="wrap muted" style="margin-bottom:40px; text-align:center; font-size:0.9em">
  © {{ year }} Isaac B. Michael<br>
  <a href="mailto:isaac.b.michael@gmail.com" target="_blank">Email</a> •
  <a href="https://www.linkedin.com/in/isaacbmichael" target="_blank">LinkedIn</a> •
  <a href="https://isaacbmichael.github.io/" target="_blank">GitHub</a>
</div>
</div></body></html>
""")

html = html_template.render(
    today=today, year=datetime.now().year, benchmark=benchmark,
    chart_base64=chart_base64, summary_html=summary_html,
    methodology_html=methodology_html,
    asset_universe_html=asset_universe_html,
    strategy_definitions_html=strategy_definitions_html
)

with open("capital_allocation_report.html", "w", encoding="utf-8") as f:
    f.write(html)
print("✅ Executive HTML report written.")

# --- Save as PDF with Playwright ---
async def html_to_pdf(input_file, output_file):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True, args=["--no-sandbox"])
        page = await browser.new_page()
        url = "file://" + os.path.abspath(input_file)
        await page.goto(url, wait_until="networkidle")
        await page.pdf(
            path=output_file,
            print_background=True,
            prefer_css_page_size=True,             # respect @page from CSS
            margin={"top": "16mm", "right": "12mm", "bottom": "18mm", "left": "12mm"}
        )
        await browser.close()

pdf_file = "capital_allocation_report.pdf"
await html_to_pdf("capital_allocation_report.html", pdf_file)
print("✅ Executive PDF report written.")


Installing dependencies...
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entr

[*********************100%***********************]  10 of 10 completed


✅ Executive HTML report written.
✅ Executive PDF report written.
